In [13]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import RegressorChain
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

from pathlib import Path

data_dir = Path('../store-sales-forecasting/data')
store_sales = pd.read_csv(
    data_dir / 'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr':'category',
        'family':'category',
        'sales':'float32',
        'onpromotion':'uint32'
    },
    parse_dates=['date'],
    infer_datetime_format=True
)

store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr','family','date']).sort_index()

family_sales = (
    store_sales
    .groupby(['family','date'])
    .mean()
    .unstack('family')
    .loc['2017']
)

test = pd.read_csv(
    data_dir / 'test.csv',
    dtype={
        'store_nbr':'category',
        'family':'category',
        'onpromotion':'uint32'
    },
    parse_dates=['date'],
    infer_datetime_format=True
)

test['date'] = test.date.dt.to_period('D')
test = test.set_index(['store_nbr','family','date']).sort_index()

C:\Users\Fikri\AppData\Local\Temp\ipykernel_3948\3804612251.py:11: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  store_sales = pd.read_csv(
C:\Users\Fikri\AppData\Local\Temp\ipykernel_3948\3804612251.py:35: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  test = pd.read_csv(


In [23]:
y = family_sales.loc[:, 'sales']

# create lags
X = pd.concat(
    {
        f'y_lag_{i}': y.shift(i)
        for i in range(1,5)
    },
    axis=1
).dropna()

#create multistep target
y = pd.concat(
    {
        f'y_step_{i+1}' : y.shift(-1)
        for i in range(16)
    },
    axis=1
).dropna()

X,y = y.align(X,join='inner',axis=0)

X.head()

y_step_1                                               
family     AUTOMOTIVE BABY CARE     BEAUTY    BEVERAGES     BOOKS   
date                                                                
2017-01-05   6.018518  0.277778   6.518518  3507.277832  0.537037  \
2017-01-06  10.259259  0.259259  10.037037  4848.518555  0.481481   
2017-01-07   9.388889  0.240741  11.611111  5503.647949  0.722222   
2017-01-08   5.944445  0.444444   5.648148  3448.203613  0.500000   
2017-01-09   4.777778  0.240741   6.500000  3171.740723  0.518519   

                                                                            
family     BREAD/BAKERY CELEBRATION     CLEANING        DAIRY        DELI   
date                                                                        
2017-01-05   513.873291   18.333334  1257.611084   864.333313  317.622406  \
2017-01-06   649.424133   21.444445  1761.351807  1173.463013  381.874756   
2017-01-07   776.717834   10.925926  1784.018555  1323.314819  429.561798   
2017-01-08   535.815979    9.240741  1208.018555   883.685181  284.767456   
2017-01-09   493.187683   15.814815  1102.685181   790.740723  256.885010   

            ... y_step_16                                          
family      ... MAGAZINES       MEATS PERSONAL CARE PET SUPPLIES   
date        ...                                                    
2017-01-05  ...  2.759259  527.617188    334.555542     7.055555  \
2017-01-06  ...  5.962963  409.145874    538.296326    11.240741   
2017-01-07  ...  5.074074  444.075836    580.574097    11.425926   
2017-01-08  ...  3.537037  306.964813    364.907410     7.851852   
2017-01-09  ...  3.222222  326.839508    334.833344     6.833333   

                                                                             
family     PLAYERS AND ELECTRONICS     POULTRY PREPARED FOODS      PRODUCE   
date                                                                         
2017-01-05               10.666667  563.664490     108.308189  2245.941406  \
2017-01-06               16.351852  499.366058     139.687485  2765.495850   
2017-01-07               18.814816  548.263184     138.670975  3002.630127   
2017-01-08               10.592592  372.827179     100.881699  2097.114014   
2017-01-09               10.092592  342.195648      99.125763  2263.040039   

                                                  
family     SCHOOL AND OFFICE SUPPLIES    SEAFOOD  
date                                              
2017-01-05                   1.018519  29.848463  
2017-01-06                   1.722222  27.406704  
2017-01-07                   2.851852  30.512369  
2017-01-08                   1.555556  20.745573  
2017-01-09                   2.203704  18.254593  

[5 rows x 528 columns]

In [22]:
X

date
2017-01-05
2017-01-06
2017-01-07
2017-01-08
2017-01-09
...
2017-08-10
2017-08-11
2017-08-12
2017-08-13
